<a href="https://colab.research.google.com/github/Hubert26/suicides_IPPAN/blob/main/data_preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
import pandas as pd
import numpy as np
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
import os
from google.colab import files

#Ustawienie braku maksymalnej ilości wyświetlanych kolumn
pd.options.display.max_columns = None

# 2023

In [124]:
df_raw_2023 = pd.read_excel('https://raw.githubusercontent.com/Hubert26/suicides_IPPAN/main/data/Samobojstwa_2023.xlsx')

In [125]:
df_raw_2023.head(5)

,ID samobójcy,Data rejestracji,Przedział wiekowy,Płeć,Stan cywilny,Wykształcenie,Informacje o pracy i nauce,Źródło utrzymania,Czy samobójstwo zakończyło się zgonem,Miejsce zamachu,Klasa miejscowości,Sposób popełnienia,Powód zamachu *,Powód zamachu 2,Powód zamachu 3,Powód zamachu 4,Stan świadomości *,Informacje dotyczące stanu zdrowia *,Informacje dotyczące leczenia z powodu alkoholizmu/narkomanii,W ciągu ostatniego miesiąca sprawca zdarzenia miał przynajmniej jeden raz kontakt z *
0,138421942,2023-01-01 01:15:42,25-29,Mężczyzna,Konkubent/konkubina,Brak danych/nieustalone,Bezrobotny,Brak danych/nieustalony,N,Mieszkanie/dom,Wieś,Samookaleczenie powierzchowne,Inny niewymieniony powyżej,Nieporozumienie rodzinne/przemoc w rodzinie,NaN,NaN,Pod wpływem alkoholu,Brak danych/nieustalono,NaN,Brak możliwości ustalenia
1,138421944,2023-01-01 01:32:02,55-59,Mężczyzna,Żonaty/zamężna,Brak danych/nieustalone,Brak danych/nieustalono,Brak danych/nieustalony,N,Mieszkanie/dom,Miasto o liczbie ludności 20 000 – 49 999,Powieszenie się,Nieustalony,NaN,NaN,NaN,Brak danych/nieustalony,Brak danych/nieustalono,NaN,Brak możliwości ustalenia
2,138421949,2023-01-01 01:54:15,13-18,Kobieta,Kawaler/panna,Brak danych/nieustalone,Uczeń,Na utrzymaniu innej osoby,N,Mieszkanie/dom,Wieś,Zażycie innych leków,Nieporozumienie rodzinne/przemoc w rodzinie,NaN,NaN,NaN,Trzeźwy(a),Brak danych/nieustalono,NaN,Placówką leczniczą
3,138421953,2023-01-01 02:19:35,55-59,Mężczyzna,Żonaty/zamężna,Brak danych/nieustalone,Brak danych/nieustalono,Brak danych/nieustalony,N,Mieszkanie/dom,Miasto o liczbie ludności do 19 999,Samookaleczenie powierzchowne,Nieporozumienie rodzinne/przemoc w rodzinie,NaN,NaN,NaN,Pod wpływem alkoholu,Nadużywał(a) alkoholu,NaN,Brak możliwości ustalenia
4,138421954,2023-01-01 02:32:52,19-24,Kobieta,Konkubent/konkubina,Brak danych/nieustalone,Brak danych/nieustalono,Brak danych/nieustalony,N,Mieszkanie/dom,Miasto o liczbie ludności 100 000 – 499 999,Zażycie innych leków,Konflikt z osobami spoza rodziny,NaN,NaN,NaN,Pod wpływem alkoholu,Nadużywał(a) alkoholu,NaN,Brak możliwości ustalenia


##Zamiana nazw kolumn

In [126]:
# Zamiana spacji na podkreślenia w nazwach kolumn
df_raw_2023.rename(columns=lambda x: x.replace(' ', '_'), inplace=True)


In [127]:
df_raw_2023.rename(columns={'Data_rejestracji': 'Data'}, inplace=True)

## Daty

In [128]:
# Wyodrębnienie roku i miesiąca
df_raw_2023['Data'] = pd.to_datetime(df_raw_2023['Data'])
df_raw_2023['Data_rok'] = df_raw_2023['Data'].dt.strftime('%Y')
df_raw_2023['Data_miesiąc'] = df_raw_2023['Data'].dt.strftime('%m')

In [129]:
# Połączenie roku i miesiąca w formacie MM.YYYY
df_raw_2023['Data'] = df_raw_2023['Data_miesiąc'] + '.' + df_raw_2023['Data_rok']

##Mapowanie

###Przedział_wiekowy

In [130]:
column = 'Przedział_wiekowy'
df_raw_2023.rename(columns={column: 'Wiek'}, inplace=True)

In [131]:
df_raw_2023['Wiek'].unique()

array(['25-29', '55-59', '13-18', '19-24', '30-34', '50-54', '40-44',
       '45-49', '35-39', '65-69', '75-79', '80-84', '60-64', '70-74',
       '85+', '7-12', 'Nieustalony wiek', nan], dtype=object)

###Płeć

In [132]:
df_raw_2023['Płeć'].unique()

array(['Mężczyzna', 'Kobieta', nan], dtype=object)

In [133]:
column = 'Płeć'
mapping = {
    "Kobieta": 'K',
    "Mężczyzna": 'M'
}

df_raw_2023[column] = df_raw_2023[column].map(mapping)


In [134]:
df_raw_2023['Płeć'].unique()

array(['M', 'K', nan], dtype=object)

###Stan_cywilny

In [135]:
df_raw_2023['Stan_cywilny'].unique()

array(['Konkubent/konkubina', 'Żonaty/zamężna', 'Kawaler/panna',
       'Brak danych/nieustalony', 'Rozwiedziony/rozwiedziona',
       'Wdowiec/wdowa', 'Separowany/separowana', nan], dtype=object)

In [136]:
column = 'Stan_cywilny'
mapping = {
    "Brak danych/nieustalony": np.nan,
    "Kawaler/panna": 'Samotność',
    "Konkubent/konkubina": 'Partnerstwo',
    "Żonaty/zamężna": 'Małżeństwo',
    "Separowany/separowana": 'Separacja',
    "Rozwiedziony/rozwiedziona": 'Rozwód',
    "Wdowiec/wdowa": 'Wdowieństwo'
}

df_raw_2023[column] = df_raw_2023[column].map(mapping)
df_raw_2023.rename(columns={column: 'St_cyw'}, inplace=True)


In [137]:
df_raw_2023['St_cyw'].unique()

array(['Partnerstwo', 'Małżeństwo', 'Samotność', nan, 'Rozwód',
       'Wdowieństwo', 'Separacja'], dtype=object)

###Wykształcenie

In [138]:
df_raw_2023['Wykształcenie'].unique()

array(['Brak danych/nieustalone', 'Podstawowe', 'Średnie',
       'Zasadnicze zawodowe', 'Podstawowe niepełne', 'Wyższe',
       'Gimnazjalne', 'Policealne', nan], dtype=object)

In [139]:
column = 'Wykształcenie'
mapping = {
    'Brak danych/nieustalone': np.nan,
    'Podstawowe niepełne': 'Podst_niepełne',
    'Podstawowe': 'Podst',
    'Gimnazjalne': 'Gimnazjalne',
    'Zasadnicze zawodowe': 'Zawodowe',
    'Średnie': 'Średnie',
    'Policealne': 'Policealne',
    'Wyższe': 'Wyższe'
}

df_raw_2023[column] = df_raw_2023[column].map(mapping)
df_raw_2023.rename(columns={column: 'Poz_edu'}, inplace=True)



In [140]:
df_raw_2023['Poz_edu'].unique()

array([nan, 'Podst', 'Średnie', 'Zawodowe', 'Podst_niepełne', 'Wyższe',
       'Gimnazjalne', 'Policealne'], dtype=object)

###Informacje_o_pracy_i_nauce

In [141]:
df_raw_2023['Informacje_o_pracy_i_nauce'].unique()

array(['Bezrobotny', 'Brak danych/nieustalono', 'Uczeń', 'Praca stała',
       'Rolnik', 'Praca dorywcza', 'Student',
       'Pracujący na własny rachunek/samodzielna działalność gospodarcza',
       nan], dtype=object)

In [142]:
column = 'Informacje_o_pracy_i_nauce'
mapping = {
    'Brak danych/nieustalono': np.nan,
    'Uczeń': 'Uczeń',
    'Student': 'Student',
    3: 'Umysłowa',
    4: 'Fizyczna',
    5: 'Rolnik',
    'Pracujący na własny rachunek/samodzielna działalność gospodarcza': 'Własna',
    'Praca stała': 'Stała',
    8: 'Dorywcza',
    'Bezrobotny': 'Bezrobotny'
}

df_raw_2023[column] = df_raw_2023[column].map(mapping)
df_raw_2023.rename(columns={column: 'Inf_praca'}, inplace=True)

In [143]:
df_raw_2023['Inf_praca'].unique()

array(['Bezrobotny', nan, 'Uczeń', 'Stała', 'Student', 'Własna'],
      dtype=object)

###Źródło_utrzymania

In [144]:
df_raw_2023['Źródło_utrzymania'].unique()

array(['Brak danych/nieustalony', 'Na utrzymaniu innej osoby', 'Praca',
       'Emerytura', 'Nie ma stałego źródła utrzymania', 'Renta',
       'Zasiłek/alimenty', nan], dtype=object)

In [145]:
column = 'Źródło_utrzymania'
mapping = {
    'Brak danych/nieustalony': np.nan,  # Zamiana 0 na NaN
    'Na utrzymaniu innej osoby': 'Zależny',
    'Praca': 'Praca',
    'Emerytura': 'Świadczenia',
    'Renta': 'Świadczenia',
    'Zasiłek/alimenty': 'Świadczenia',
    'Nie ma stałego źródła utrzymania': 'Bez_dochodu'
}

df_raw_2023[column] = df_raw_2023[column].map(mapping)
df_raw_2023.rename(columns={column: 'Utrzymanie'}, inplace=True)

In [146]:
df_raw_2023['Utrzymanie'].unique()

array([nan, 'Zależny', 'Praca', 'Świadczenia', 'Bez_dochodu'],
      dtype=object)

###Czy_samobójstwo_zakończyło_się_zgonem

In [147]:
df_raw_2023['Czy_samobójstwo_zakończyło_się_zgonem'].unique()

array(['N', 'T', nan], dtype=object)

In [148]:
column = 'Czy_samobójstwo_zakończyło_się_zgonem'

mapping = {
    0: np.nan,  # Zamiana 0 na NaN
    'T': 1,
    'N': 0
}
df_raw_2023[column] = df_raw_2023[column].map(mapping)
df_raw_2023.rename(columns={column: 'Sukces'}, inplace=True)

In [149]:
df_raw_2023['Sukces'].unique()

array([ 0.,  1., nan])

###Miejsce_zamachu

In [150]:
df_raw_2023['Miejsce_zamachu'].unique()

array(['Mieszkanie/dom', 'Rzeka/jezioro/inny zbiornik wodny',
       'Zabudowania gospodarcze', 'Garaż/piwnica/strych',
       'Droga/ulica/chodnik', 'Teren kolei/tory', 'Park, las', 'Inne',
       'Szkoła/uczelnia', 'Zakład pracy',
       'Placówka wychowawczo-opiekuńcza',
       'Placówka lecznicza lub sanatoryjna', 'Miejsce prawnej izolacji',
       'Obiekt policyjny', 'Obiekt wojskowy', nan], dtype=object)

In [151]:
column = 'Miejsce_zamachu'
mapping = {
    0: np.nan,
    'Droga/ulica/chodnik': 'Droga',
    'Zabudowania gospodarcze': 'Zabudowania',
    'Mieszkanie/dom': 'Mieszkanie',
    'Teren kolei/tory': 'Tory',
    'Park, las': 'Park',
    'Garaż/piwnica/strych': 'Piwnica',
    'Rzeka/jezioro/inny zbiornik wodny': 'Woda',
    'Zakład pracy': 'Praca',
    'Placówka lecznicza lub sanatoryjna': 'Lecznica',
    'Miejsce prawnej izolacji': 'Izolacja',
    'Obiekt wojskowy': 'Wojsko',
    'Placówka wychowawczo-opiekuńcza': 'Opieka',
    'Szkoła/uczelnia': 'Szkoła',
    'Obiekt policyjny': 'Policja',
    'Inne': 'Inne'
}

df_raw_2023[column] = df_raw_2023[column].map(mapping)
df_raw_2023.rename(columns={column: 'Miejsce'}, inplace=True)


In [152]:
df_raw['Miejsce'].unique()

array(['Mieszkanie', 'Woda', 'Zabudowania', 'Piwnica', 'Droga', 'Tory',
       'Park', 'Inne', 'Szkoła', 'Praca', 'Opieka', 'Lecznica',
       'Izolacja', 'Policja', 'Wojsko', nan], dtype=object)

###Sposób_popełnienia

In [153]:
df_raw_2023['Sposób_popełnienia'].unique()

array(['Samookaleczenie powierzchowne', 'Powieszenie się',
       'Zażycie innych leków',
       'Zażycie środków nasennych/leków psychotropowych',
       'Uszkodzenie układu krwionośnego', 'Inny',
       'Utonięcie/utopienie się', 'Rzucenie się pod pojazd w ruchu',
       'Uduszenie się', 'Rzucenie się z wysokości',
       'Zastrzelenie się/użycie broni palnej', 'Zatrucie gazem/spalinami',
       'Samopodpalenie', 'Zatrucie środkami chemicznymi/toksycznymi',
       'Zatrucie środkami odurzającymi', 'Zatrucie dopalaczami', nan],
      dtype=object)

In [154]:
column = 'Sposób_popełnienia'
mapping = {
    0: np.nan,
    'Rzucenie się pod pojazd w ruchu': 'Pod_pojazd',
    'Rzucenie się z wysokości': 'Z_wysokości',
    'Powieszenie się': 'Powieszenie',
    'Uszkodzenie układu krwionośnego': 'Układ_krwionośny',
    'Zastrzelenie się/użycie broni palnej': 'Zastrzelenie',
    'Samookaleczenie powierzchowne': 'Samookaleczenie_pow.',
    7: 'Samookaleczenie_inne',
    'Zażycie środków nasennych/leków psychotropowych': 'Środki_nasenne',
    9: 'Trucizna',
    'Zatrucie gazem/spalinami': 'Gaz',
    'Zażycie innych leków': 'Inne_leki',
    'Zatrucie środkami chemicznymi/toksycznymi': 'Środki_chemiczne',
    'Zatrucie środkami odurzającymi': 'Odurzające',
    'Zatrucie dopalaczami': 'Dopalacze',
    'Utonięcie/utopienie się': 'Utonięcie',
    'Samopodpalenie': 'Podpalenie',
    'Uduszenie się': 'Uduszenie',
    'Inny': 'Inny'
}

df_raw_2023[column] = df_raw_2023[column].map(mapping)
df_raw_2023.rename(columns={column: 'Sposób'}, inplace=True)

In [155]:
df_raw_2023['Sposób'].unique()

array(['Samookaleczenie_pow.', 'Powieszenie', 'Inne_leki',
       'Środki_nasenne', 'Układ_krwionośny', 'Inny', 'Utonięcie',
       'Pod_pojazd', 'Uduszenie', 'Z_wysokości', 'Zastrzelenie', 'Gaz',
       'Podpalenie', 'Środki_chemiczne', 'Odurzające', 'Dopalacze', nan],
      dtype=object)

###Powód_zamachu

In [156]:
list(set(df_raw_2023['Powód_zamachu_*'].unique()) | set(df_raw_2023['Powód_zamachu_2'].unique()) | set(df_raw_2023['Powód_zamachu_3'].unique()) | set(df_raw_2023['Powód_zamachu_4'].unique()))

['Nagła utrata źródła utrzymania',
 'Dokonanie przestępstwa lub wykroczenia',
 'Mobbing, cybermobbing, znęcanie',
 'Inny niewymieniony powyżej',
 'Leczony(a) psychiatrycznie',
 'Nieporozumienie rodzinne/przemoc w rodzinie',
 'Niepożądana ciąża',
 nan,
 'Pogorszenie lub nagła utrata zdrowia',
 'Problemy w szkole lub pracy',
 'Śmierć bliskiej osoby',
 'Zagrożenie lub utrata miejsca zamieszkania',
 'Złe warunki ekonomiczne/długi',
 'Choroba fizyczna',
 'Choroba psychiczna/zaburzenia psychiczne',
 'Trwałe kalectwo',
 'Zawód miłosny',
 'Nieustalony',
 'Konflikt z osobami spoza rodziny',
 'Nosiciel wirusa HIV, chory na AIDS']

In [157]:
column = 'Powód_zamachu_*'
mapping = {
    'Nieustalony': np.nan,
    'Zawód miłosny': 'Miłość',
    'Leczony(a) psychiatrycznie': 'Choroby_psych',
    'Nieporozumienie rodzinne/przemoc w rodzinie': 'Konflikty_rodzinne',
    'Nosiciel wirusa HIV, chory na AIDS': 'HIV/AIDS',
    'Nagła utrata źródła utrzymania': 'Utrata_utrzymania',
    'Złe warunki ekonomiczne/długi': 'Finanse',
    'Choroba psychiczna/zaburzenia psychiczne': 'Choroby_psych',
    'Problemy w szkole lub pracy': 'Praca',
    'Śmierć bliskiej osoby': 'Śmierć_bliskiego',
    'Dokonanie przestępstwa lub wykroczenia': 'Przestępstwa',
    'Trwałe kalectwo': 'Kalectwo',
    'Niepożądana ciąża': 'Ciąża',
    'Choroba fizyczna': 'Choroby_fiz',
    'Pogorszenie lub nagła utrata zdrowia': 'Zdrowie',
    'Konflikt z osobami spoza rodziny': 'Konflikty_inni',
    'Zagrożenie lub utrata miejsca zamieszkania': 'Bezdomność',
    'Mobbing, cybermobbing, znęcanie': 'Mobbing',
    'Inny niewymieniony powyżej': 'Inne'
}

df_raw_2023[column] = df_raw_2023[column].map(mapping)
df_raw_2023.rename(columns={column: 'Powód'}, inplace=True)

###Powód_zamachu_2

In [158]:
column = 'Powód_zamachu_2'
mapping = {
    'Nieustalony': np.nan,
    'Zawód miłosny': 'Miłość',
    'Leczony(a) psychiatrycznie': 'Choroby_psych',
    'Nieporozumienie rodzinne/przemoc w rodzinie': 'Konflikty_rodzinne',
    'Nosiciel wirusa HIV, chory na AIDS': 'HIV/AIDS',
    'Nagła utrata źródła utrzymania': 'Utrata_utrzymania',
    'Złe warunki ekonomiczne/długi': 'Finanse',
    'Choroba psychiczna/zaburzenia psychiczne': 'Choroby_psych',
    'Problemy w szkole lub pracy': 'Praca',
    'Śmierć bliskiej osoby': 'Śmierć_bliskiego',
    'Dokonanie przestępstwa lub wykroczenia': 'Przestępstwa',
    'Trwałe kalectwo': 'Kalectwo',
    'Niepożądana ciąża': 'Ciąża',
    'Choroba fizyczna': 'Choroby_fiz',
    'Pogorszenie lub nagła utrata zdrowia': 'Zdrowie',
    'Konflikt z osobami spoza rodziny': 'Konflikty_inni',
    'Zagrożenie lub utrata miejsca zamieszkania': 'Bezdomność',
    'Mobbing, cybermobbing, znęcanie': 'Mobbing',
    'Inny niewymieniony powyżej': 'Inne'
}

df_raw_2023[column] = df_raw_2023[column].map(mapping)
df_raw_2023.rename(columns={column: 'Powód_2'}, inplace=True)

###Powód_zamachu_3

In [159]:
column = 'Powód_zamachu_3'
mapping = {
    'Nieustalony': np.nan,
    'Zawód miłosny': 'Miłość',
    'Leczony(a) psychiatrycznie': 'Choroby_psych',
    'Nieporozumienie rodzinne/przemoc w rodzinie': 'Konflikty_rodzinne',
    'Nosiciel wirusa HIV, chory na AIDS': 'HIV/AIDS',
    'Nagła utrata źródła utrzymania': 'Utrata_utrzymania',
    'Złe warunki ekonomiczne/długi': 'Finanse',
    'Choroba psychiczna/zaburzenia psychiczne': 'Choroby_psych',
    'Problemy w szkole lub pracy': 'Praca',
    'Śmierć bliskiej osoby': 'Śmierć_bliskiego',
    'Dokonanie przestępstwa lub wykroczenia': 'Przestępstwa',
    'Trwałe kalectwo': 'Kalectwo',
    'Niepożądana ciąża': 'Ciąża',
    'Choroba fizyczna': 'Choroby_fiz',
    'Pogorszenie lub nagła utrata zdrowia': 'Zdrowie',
    'Konflikt z osobami spoza rodziny': 'Konflikty_inni',
    'Zagrożenie lub utrata miejsca zamieszkania': 'Bezdomność',
    'Mobbing, cybermobbing, znęcanie': 'Mobbing',
    'Inny niewymieniony powyżej': 'Inne'
}

df_raw_2023[column] = df_raw_2023[column].map(mapping)
df_raw_2023.rename(columns={column: 'Powód_3'}, inplace=True)

###Powód_zamachu_4

In [160]:
column = 'Powód_zamachu_4'
mapping = {
    'Nieustalony': np.nan,
    'Zawód miłosny': 'Miłość',
    'Leczony(a) psychiatrycznie': 'Choroby_psych',
    'Nieporozumienie rodzinne/przemoc w rodzinie': 'Konflikty_rodzinne',
    'Nosiciel wirusa HIV, chory na AIDS': 'HIV/AIDS',
    'Nagła utrata źródła utrzymania': 'Utrata_utrzymania',
    'Złe warunki ekonomiczne/długi': 'Finanse',
    'Choroba psychiczna/zaburzenia psychiczne': 'Choroby_psych',
    'Problemy w szkole lub pracy': 'Praca',
    'Śmierć bliskiej osoby': 'Śmierć_bliskiego',
    'Dokonanie przestępstwa lub wykroczenia': 'Przestępstwa',
    'Trwałe kalectwo': 'Kalectwo',
    'Niepożądana ciąża': 'Ciąża',
    'Choroba fizyczna': 'Choroby_fiz',
    'Pogorszenie lub nagła utrata zdrowia': 'Zdrowie',
    'Konflikt z osobami spoza rodziny': 'Konflikty_inni',
    'Zagrożenie lub utrata miejsca zamieszkania': 'Bezdomność',
    'Mobbing, cybermobbing, znęcanie': 'Mobbing',
    'Inny niewymieniony powyżej': 'Inne'
}

df_raw_2023[column] = df_raw_2023[column].map(mapping)
df_raw_2023.rename(columns={column: 'Powód_4'}, inplace=True)

In [161]:
list(set(df_raw_2023['Powód'].unique()) | set(df_raw_2023['Powód_2'].unique()) | set(df_raw_2023['Powód_3'].unique()) | set(df_raw_2023['Powód_4'].unique()))

['HIV/AIDS',
 'Śmierć_bliskiego',
 'Bezdomność',
 nan,
 'Zdrowie',
 'Praca',
 'Miłość',
 'Konflikty_inni',
 'Utrata_utrzymania',
 'Choroby_fiz',
 'Kalectwo',
 'Inne',
 'Finanse',
 'Choroby_psych',
 'Przestępstwa',
 'Mobbing',
 'Ciąża',
 'Konflikty_rodzinne']

###Stan_świadomości

In [162]:
df_raw_2023['Stan_świadomości_*'].unique()

array(['Pod wpływem alkoholu', 'Brak danych/nieustalony', 'Trzeźwy(a)',
       'Pod wpływem leków', 'Pod wpływem alkoholu i środków odurzających',
       'Pod wpływem alkoholu i leków', 'Pod wpływem środków odurzających',
       'Pod wpływem alkoholu, leków i środków odurzających',
       'Pod wpływem zastępczych środków/substancji (dopalaczy)',
       'Pod wpływem alkoholu i zastępczych środków/substancji (dopalaczy)',
       'Pod wpływem leków i środków odurzających',
       'Pod wpływem alkoholu zastępczych środków/substancji (dopalaczy)',
       'Pod wpływem leków i srodków odurzających', nan], dtype=object)

In [163]:
column = 'Stan_świadomości_*'
mapping = {
    'Brak danych/nieustalony': np.nan,
    'Trzeźwy(a)': 'Trzeźwy',
    'Pod wpływem alkoholu': 'Alko',
    'Pod wpływem zastępczych środków/substancji (dopalaczy)': 'Dopa',
    4: 'Narko',
    'Pod wpływem leków': 'Leki',
    'Pod wpływem środków odurzających': 'Odurza',
    'Pod wpływem alkoholu i zastępczych środków/substancji (dopalaczy)': 'Alko+dopa',
    'Pod wpływem alkoholu zastępczych środków/substancji (dopalaczy)': 'Alko+dopa',
    8: 'Alko+narko',
    'Pod wpływem alkoholu i leków': 'Alko+leki',
    'Pod wpływem alkoholu i środków odurzających': 'Alko+odurza',
    11: 'Alko+dopa+odurza',
    12: 'Dopa+odurza',
    'Pod wpływem leków i środków odurzających': 'Leki+odurza',
    14: 'Leki+dopa',
    15: 'Leki+narko',
    16: 'Leki+dopa+odurza',
    'Pod wpływem alkoholu, leków i środków odurzających': 'Alko+leki+odurza',
    18: 'Alko+leki+dopa+odurza',
}

df_raw_2023[column] = df_raw_2023[column].map(mapping)
df_raw_2023.rename(columns={column: 'Stan'}, inplace=True)

In [164]:
df_raw_2023['Stan'].unique()

array(['Alko', nan, 'Trzeźwy', 'Leki', 'Alko+odurza', 'Alko+leki',
       'Odurza', 'Alko+leki+odurza', 'Dopa', 'Alko+dopa', 'Leki+odurza'],
      dtype=object)

###Informacje_dotyczące_leczenia_z_powodu_alkoholizmu/narkomanii

In [165]:
df_raw_2023['Informacje_dotyczące_leczenia_z_powodu_alkoholizmu/narkomanii'].unique()

array([nan, 'Nadużywał(a) alkoholu', 'Leczony(a) z powodu alkoholizmu',
       'Leczony(a) z powodu narkomanii',
       'Leczony(a) z powodu alkoholizmu i narkomanii',
       'Leczony(a) psychiatrycznie'], dtype=object)

In [166]:
column = 'Informacje_dotyczące_leczenia_z_powodu_alkoholizmu/narkomanii'
mapping = {
    'Leczony(a) psychiatrycznie': np.nan,
    'Nadużywał(a) alkoholu': 'Alko',
    'Leczony(a) z powodu alkoholizmu': 'Alko',
    'Leczony(a) z powodu narkomanii': 'Narko',
    'Leczony(a) z powodu alkoholizmu i narkomanii': 'Alko+narko'
}

df_raw_2023[column] = df_raw_2023[column].map(mapping)
df_raw_2023.rename(columns={column: 'Leczenie'}, inplace=True)

In [167]:
df_raw_2023['Leczenie'].unique()

array([nan, 'Alko', 'Narko', 'Alko+narko'], dtype=object)

##Powód zamachu - dummies

In [168]:
columns_to_drop = ['Powód', 'Powód_2', 'Powód_3', 'Powód_4']
df_powód_zamachu_2023 = df_raw_2023[columns_to_drop]
df_raw_2023.drop(columns=columns_to_drop, inplace=True)


In [169]:
powód_zamachu_values = (
    set(df_powód_zamachu_2023['Powód'].unique()) |
    set(df_powód_zamachu_2023['Powód_2'].unique()) |
    set(df_powód_zamachu_2023['Powód_3'].unique()) |
    set(df_powód_zamachu_2023['Powód_4'].unique())
)

In [170]:
powód_zamachu_values

{'Bezdomność',
 'Choroby_fiz',
 'Choroby_psych',
 'Ciąża',
 'Finanse',
 'HIV/AIDS',
 'Inne',
 'Kalectwo',
 'Konflikty_inni',
 'Konflikty_rodzinne',
 'Miłość',
 'Mobbing',
 'Praca',
 'Przestępstwa',
 'Utrata_utrzymania',
 'Zdrowie',
 nan,
 'Śmierć_bliskiego'}

In [171]:
for value in powód_zamachu_values:
    column_name = 'Powód_' + str(value)
    df_powód_zamachu_2023[column_name] = df_powód_zamachu_2023.apply(lambda row: 1 if value in row.values else 0, axis=1)


<ipython-input-171-23ef7d7d8362>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_powód_zamachu_2023[column_name] = df_powód_zamachu_2023.apply(lambda row: 1 if value in row.values else 0, axis=1)
<ipython-input-171-23ef7d7d8362>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_powód_zamachu_2023[column_name] = df_powód_zamachu_2023.apply(lambda row: 1 if value in row.values else 0, axis=1)
<ipython-input-171-23ef7d7d8362>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fr

In [172]:
columns_to_drop = ['Powód', 'Powód_2', 'Powód_3', 'Powód_4', 'Powód_nan']
df_powód_zamachu_2023.drop(columns=columns_to_drop, inplace=True)


In [175]:
df_powód_zamachu_2023.head(5)

,Powód_HIV/AIDS,Powód_Śmierć_bliskiego,Powód_Bezdomność,Powód_Zdrowie,Powód_Praca,Powód_Miłość,Powód_Konflikty_inni,Powód_Utrata_utrzymania,Powód_Choroby_fiz,Powód_Kalectwo,Powód_Inne,Powód_Finanse,Powód_Choroby_psych,Powód_Przestępstwa,Powód_Mobbing,Powód_Ciąża,Powód_Konflikty_rodzinne
0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


##Liczba_powodów_zamachu

In [173]:
df_raw_2023['L_powodów'] = df_powód_zamachu_2023.sum(axis=1)

In [174]:
df_raw_2023.head(5)

,ID_samobójcy,Data,Wiek,Płeć,St_cyw,Poz_edu,Inf_praca,Utrzymanie,Sukces,Miejsce,Klasa_miejscowości,Sposób,Stan,Informacje_dotyczące_stanu_zdrowia_*,Leczenie,W_ciągu_ostatniego_miesiąca_sprawca_zdarzenia_miał_przynajmniej_jeden_raz_kontakt_z_*,Data_rok,Data_miesiąc,L_powodów
0,138421942,01.2023,25-29,M,Partnerstwo,NaN,Bezrobotny,NaN,0.0,Mieszkanie,Wieś,Samookaleczenie_pow.,Alko,Brak danych/nieustalono,NaN,Brak możliwości ustalenia,2023,01,2
1,138421944,01.2023,55-59,M,Małżeństwo,NaN,NaN,NaN,0.0,Mieszkanie,Miasto o liczbie ludności 20 000 – 49 999,Powieszenie,NaN,Brak danych/nieustalono,NaN,Brak możliwości ustalenia,2023,01,0
2,138421949,01.2023,13-18,K,Samotność,NaN,Uczeń,Zależny,0.0,Mieszkanie,Wieś,Inne_leki,Trzeźwy,Brak danych/nieustalono,NaN,Placówką leczniczą,2023,01,1
3,138421953,01.2023,55-59,M,Małżeństwo,NaN,NaN,NaN,0.0,Mieszkanie,Miasto o liczbie ludności do 19 999,Samookaleczenie_pow.,Alko,Nadużywał(a) alkoholu,NaN,Brak możliwości ustalenia,2023,01,1
4,138421954,01.2023,19-24,K,Partnerstwo,NaN,NaN,NaN,0.0,Mieszkanie,Miasto o liczbie ludności 100 000 – 499 999,Inne_leki,Alko,Nadużywał(a) alkoholu,NaN,Brak możliwości ustalenia,2023,01,1


# 2013_2022

In [176]:
df_raw_2013_2022 = pd.read_csv('https://raw.githubusercontent.com/Hubert26/suicides_IPPAN/main/data/final_samobojstwa_2013_2022.csv', delimiter=',', low_memory=False)

In [177]:
df_raw_2013_2022.head(5)

,ID samobójcy,Data raportu [RRRRMM],Przedział wiekowy,Płeć,Stan cywilny,Wykształcenie,Informacje o pracy i nauce,Źródło utrzymania,Czy samobójstwo zakończyło się zgonem,Miejsce zamachu,Sposób popełnienia,Powód zamachu,Powód zamachu 2,Powód zamachu 3,Powód zamachu 4,Stan świadomości,Informacje o używaniu substancji,Informacje dotyczące leczenia z powodu alkoholizmu/narkomanii
0,"114581552,00",201301,0,2,0,0,0,0,0,0,0,0,NaN,NaN,NaN,0,0,0
1,"114584284,00",201301,2,2,0,0,1,0,1,1,1,0,NaN,NaN,NaN,0,0,0
2,"114584293,00",201301,2,1,1,2,1,1,1,15,2,0,NaN,NaN,NaN,0,0,0
3,"114584312,00",201301,3,2,1,0,0,0,1,0,18,1,NaN,NaN,NaN,0,0,0
4,"114584399,00",201301,7,2,1,0,9,0,1,2,3,0,NaN,NaN,NaN,2,0,0


##Zamiana nazw kolumn

In [178]:
# Zamiana spacji na podkreślenia w nazwach kolumn
df_raw_2013_2022.rename(columns=lambda x: x.replace(' ', '_'), inplace=True)


In [179]:
df_raw_2013_2022.rename(columns={'Data_raportu_[RRRRMM]': 'Data'}, inplace=True)

## Daty

In [180]:
# Wyodrębnienie roku i miesiąca
df_raw_2013_2022['Data_rok'] = df_raw_2013_2022['Data'].apply(lambda x: x[0:4] if len(x) > 8 else x[:4])
df_raw_2013_2022['Data_miesiąc'] = df_raw_2013_2022['Data'].apply(lambda x: x[5:7] if len(x) > 8 else x[4:6])

In [181]:
# Połączenie roku i miesiąca w formacie MM.YYYY
df_raw_2013_2022['Data'] = df_raw_2013_2022['Data_miesiąc'] + '.' + df_raw_2013_2022['Data_rok']

##Mapowanie

###Przedział_wiekowy

In [183]:
column = 'Przedział_wiekowy'
mapping = {
    0: np.nan,
    1: '07-12',
    2: '13-18',
    3: '19-24',
    4: '25-29',
    5: '30-34',
    6: '35-39',
    7: '40-44',
    8: '45-49',
    9: '50-54',
    10: '55-59',
    11: '60-64',
    12: '65-69',
    13: '70-74',
    14: '75-79',
    15: '80-84',
    16: '85+'
}

df_raw_2013_2022[column] = df_raw_2013_2022[column].map(mapping)
df_raw_2013_2022.rename(columns={column: 'Wiek'}, inplace=True)

###Płeć

In [184]:
column = 'Płeć'
mapping = {
    0: np.nan,
    1: 'K',
    2: 'M'
}

df_raw_2013_2022[column] = df_raw_2013_2022[column].map(mapping)


###Stan_cywilny

In [185]:
column = 'Stan_cywilny'
mapping = {
    0: np.nan,
    1: 'Samotność',
    2: 'Partnerstwo',
    3: 'Małżeństwo',
    4: 'Separacja',
    5: 'Rozwód',
    6: 'Wdowieństwo'
}

df_raw_2013_2022[column] = df_raw_2013_2022[column].map(mapping)
df_raw_2013_2022.rename(columns={column: 'St_cyw'}, inplace=True)


###Wykształcenie

In [186]:
column = 'Wykształcenie'
mapping = {
    0: np.nan,
    1: 'Podst_niepełne',
    2: 'Podst',
    3: 'Gimnazjalne',
    4: 'Zawodowe',
    5: 'Średnie',
    6: 'Policealne',
    7: 'Wyższe'
}

df_raw_2013_2022[column] = df_raw_2013_2022[column].map(mapping)
df_raw_2013_2022.rename(columns={column: 'Poz_edu'}, inplace=True)



###Informacje_o_pracy_i_nauce

In [187]:
column = 'Informacje_o_pracy_i_nauce'
mapping = {
    0: np.nan,
    1: 'Uczeń',
    2: 'Student',
    3: 'Umysłowa',
    4: 'Fizyczna',
    5: 'Rolnik',
    6: 'Własna',
    7: 'Stała',
    8: 'Dorywcza',
    9: 'Bezrobotny'
}

df_raw_2013_2022[column] = df_raw_2013_2022[column].map(mapping)
df_raw_2013_2022.rename(columns={column: 'Inf_praca'}, inplace=True)

###Źródło_utrzymania

In [188]:
column = 'Źródło_utrzymania'
mapping = {
    0: np.nan,  # Zamiana 0 na NaN
    1: 'Zależny',
    2: 'Praca',
    3: 'Świadczenia',
    4: 'Bez_dochodu'
}

df_raw_2013_2022[column] = df_raw_2013_2022[column].map(mapping)
df_raw_2013_2022.rename(columns={column: 'Utrzymanie'}, inplace=True)

###Czy_samobójstwo_zakończyło_się_zgonem

In [189]:
column = 'Czy_samobójstwo_zakończyło_się_zgonem'

mapping = {
    0: np.nan,  # Zamiana 0 na NaN
    1: 1,
    2: 0
}
df_raw_2013_2022[column] = df_raw_2013_2022[column].map(mapping)
df_raw_2013_2022.rename(columns={column: 'Sukces'}, inplace=True)

###Miejsce_zamachu

In [190]:
column = 'Miejsce_zamachu'
mapping = {
    0: np.nan,
    1: 'Droga',
    2: 'Zabudowania',
    3: 'Mieszkanie',
    4: 'Tory',
    5: 'Park',
    6: 'Piwnica',
    7: 'Woda',
    8: 'Praca',
    9: 'Lecznica',
    10: 'Izolacja',
    11: 'Wojsko',
    12: 'Opieka',
    13: 'Szkoła',
    14: 'Policja',
    15: 'Inne'
}

df_raw_2013_2022[column] = df_raw_2013_2022[column].map(mapping)
df_raw_2013_2022.rename(columns={column: 'Miejsce'}, inplace=True)


###Sposób_popełnienia

In [191]:
column = 'Sposób_popełnienia'
mapping = {
    0: np.nan,
    1: 'Pod_pojazd',
    2: 'Z_wysokości',
    3: 'Powieszenie',
    4: 'Układ_krwionośny',
    5: 'Zastrzelenie',
    6: 'Samookaleczenie_pow.',
    7: 'Samookaleczenie_inne',
    8: 'Środki_nasenne',
    9: 'Trucizna',
    10: 'Gaz',
    11: 'Inne_leki',
    12: 'Środki_chemiczne',
    13: 'Odurzające',
    14: 'Dopalacze',
    15: 'Utonięcie',
    16: 'Podpalenie',
    17: 'Uduszenie',
    18: 'Inny'
}

df_raw_2013_2022[column] = df_raw_2013_2022[column].map(mapping)
df_raw_2013_2022.rename(columns={column: 'Sposób'}, inplace=True)

###Powód_zamachu

In [192]:
column = 'Powód_zamachu'
mapping = {
    0: np.nan,
    1: 'Miłość',
    2: 'Choroby_psych',
    3: 'Konflikty_rodzinne',
    4: 'HIV/AIDS',
    5: 'Utrata_utrzymania',
    6: 'Finanse',
    7: 'Choroba',
    8: 'Praca',
    9: 'Śmierć_bliskiego',
    10: 'Przestępstwa',
    11: 'Kalectwo',
    12: 'Ciąża',
    13: 'Choroby_fiz',
    14: 'Zdrowie',
    15: 'Konflikty_inni',
    16: 'Bezdomność',
    17: 'Mobbing',
    18: 'Inne'
}

df_raw_2013_2022[column] = df_raw_2013_2022[column].map(mapping)
df_raw_2013_2022.rename(columns={column: 'Powód'}, inplace=True)

###Powód_zamachu_2

In [193]:
column = 'Powód_zamachu_2'
mapping = {
    0: np.nan,
    1: 'Miłość',
    2: 'Choroby_psych',
    3: 'Konflikty_rodzinne',
    4: 'HIV/AIDS',
    5: 'Utrata_utrzymania',
    6: 'Finanse',
    7: 'Choroba',
    8: 'Praca',
    9: 'Śmierć_bliskiego',
    10: 'Przestępstwa',
    11: 'Kalectwo',
    12: 'Ciąża',
    13: 'Choroby_fiz',
    14: 'Zdrowie',
    15: 'Konflikty_inni',
    16: 'Bezdomność',
    17: 'Mobbing',
    18: 'Inne'
}

df_raw_2013_2022[column] = df_raw_2013_2022[column].map(mapping)
df_raw_2013_2022.rename(columns={column: 'Powód_2'}, inplace=True)

###Powód_zamachu_3

In [194]:
column = 'Powód_zamachu_3'
mapping = {
    0: np.nan,
    1: 'Miłość',
    2: 'Choroby_psych',
    3: 'Konflikty_rodzinne',
    4: 'HIV/AIDS',
    5: 'Utrata_utrzymania',
    6: 'Finanse',
    7: 'Choroba',
    8: 'Praca',
    9: 'Śmierć_bliskiego',
    10: 'Przestępstwa',
    11: 'Kalectwo',
    12: 'Ciąża',
    13: 'Choroby_fiz',
    14: 'Zdrowie',
    15: 'Konflikty_inni',
    16: 'Bezdomność',
    17: 'Mobbing',
    18: 'Inne'
}

df_raw_2013_2022[column] = df_raw_2013_2022[column].map(mapping)
df_raw_2013_2022.rename(columns={column: 'Powód_3'}, inplace=True)

###Powód_zamachu_4

In [195]:
column = 'Powód_zamachu_4'
mapping = {
    0: np.nan,
    1: 'Miłość',
    2: 'Choroby_psych',
    3: 'Konflikty_rodzinne',
    4: 'HIV/AIDS',
    5: 'Utrata_utrzymania',
    6: 'Finanse',
    7: 'Choroba',
    8: 'Praca',
    9: 'Śmierć_bliskiego',
    10: 'Przestępstwa',
    11: 'Kalectwo',
    12: 'Ciąża',
    13: 'Choroby_fiz',
    14: 'Zdrowie',
    15: 'Konflikty_inni',
    16: 'Bezdomność',
    17: 'Mobbing',
    18: 'Inne'
}

df_raw_2013_2022[column] = df_raw_2013_2022[column].map(mapping)
df_raw_2013_2022.rename(columns={column: 'Powód_4'}, inplace=True)

###Stan_świadomości

In [196]:
column = 'Stan_świadomości'
mapping = {
    0: np.nan,
    1: 'Trzeźwy',
    2: 'Alko',
    3: 'Dopa',
    4: 'Narko',
    5: 'Leki',
    6: 'Odurza',
    7: 'Alko+dopa',
    8: 'Alko+narko',
    9: 'Alko+leki',
    10: 'Alko+odurza',
    11: 'Alko+dopa+odurza',
    12: 'Dopa+odurza',
    13: 'Leki+odurza',
    14: 'Leki+dopa',
    15: 'Leki+narko',
    16: 'Leki+dopa+odurza',
    17: 'Alko+leki+odurza',
    18: 'Alko+leki+dopa+odurza'
}

df_raw_2013_2022[column] = df_raw_2013_2022[column].map(mapping)
df_raw_2013_2022.rename(columns={column: 'Stan'}, inplace=True)

###Informacje_o_używaniu_substancji

In [197]:
column = 'Informacje_o_używaniu_substancji'
mapping = {
    0: np.nan,
    1: 'Alko',
    2: 'Narko',
    3: 'Dopa',
    4: 'Alko+dopa',
    5: 'Narko+dopa',
    6: 'Alko+narko',
    7: 'Alko+narko+dopa'
}

df_raw_2013_2022[column] = df_raw_2013_2022[column].map(mapping)
df_raw_2013_2022.rename(columns={column: 'Używanie'}, inplace=True)


###Informacje_dotyczące_leczenia_z_powodu_alkoholizmu/narkomanii

In [198]:
column = 'Informacje_dotyczące_leczenia_z_powodu_alkoholizmu/narkomanii'
mapping = {
    0: np.nan,
    1: 'Alko',
    2: 'Narko',
    3: 'Alko+narko'
}

df_raw_2013_2022[column] = df_raw_2013_2022[column].map(mapping)
df_raw_2013_2022.rename(columns={column: 'Leczenie'}, inplace=True)

##Powód zamachu - dummies

In [ ]:
columns_to_drop = ['Powód', 'Powód_2', 'Powód_3', 'Powód_4']
df_powód_zamachu_2013_2022 = df_raw_2013_2022[columns_to_drop]
df_raw_2013_2022.drop(columns=columns_to_drop, inplace=True)


In [ ]:
powód_zamachu_values = (
    set(df_powód_zamachu_2013_2022['Powód'].unique()) |
    set(df_powód_zamachu_2013_2022['Powód_2'].unique()) |
    set(df_powód_zamachu_2013_2022['Powód_3'].unique()) |
    set(df_powód_zamachu_2013_2022['Powód_4'].unique())
)

In [ ]:
powód_zamachu_values

{'Bezdomność',
 'Choroba',
 'Choroby_fiz',
 'Choroby_psych',
 'Ciąża',
 'Finanse',
 'HIV/AIDS',
 'Inne',
 'Kalectwo',
 'Konflikty_inni',
 'Konflikty_rodzinne',
 'Miłość',
 'Mobbing',
 'Praca',
 'Przestępstwa',
 'Utrata_utrzymania',
 'Zdrowie',
 nan,
 'Śmierć_bliskiego'}

In [ ]:
for value in powód_zamachu_values:
    column_name = 'Powód_' + str(value)
    df_powód_zamachu_2013_2022[column_name] = df_powód_zamachu_2013_2022.apply(lambda row: 1 if value in row.values else 0, axis=1)


<ipython-input-27-9a28d6c29fb4>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_powód_zamachu[column_name] = df_powód_zamachu.apply(lambda row: 1 if value in row.values else 0, axis=1)
<ipython-input-27-9a28d6c29fb4>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_powód_zamachu[column_name] = df_powód_zamachu.apply(lambda row: 1 if value in row.values else 0, axis=1)
<ipython-input-27-9a28d6c29fb4>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usi

In [ ]:
columns_to_drop = ['Powód', 'Powód_2', 'Powód_3', 'Powód_4', 'Powód_nan']
df_powód_zamachu_2013_2022.drop(columns=columns_to_drop, inplace=True)


##Liczba_powodów_zamachu

In [ ]:
df_raw_2013_2022['L_powodów'] = df_powód_zamachu_2013_2022.sum(axis=1)

In [ ]:
df_raw_2013_2022.head(5)

,ID_samobójcy,Data,Wiek,Płeć,St_cyw,Poz_edu,Inf_praca,Utrzymanie,Sukces,Miejsce,Sposób,Stan,Używanie,Leczenie,Data_rok,Data_miesiąc,L_powodów
0,"114581552,00",01.2013,NaN,M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013,01,0
1,"114584284,00",01.2013,13-18,M,NaN,NaN,Uczeń,NaN,1.0,Droga,Pod_pojazd,NaN,NaN,NaN,2013,01,0
2,"114584293,00",01.2013,13-18,K,Samotność,Podst,Uczeń,Zależny,1.0,Inne,Z_wysokości,NaN,NaN,NaN,2013,01,0
3,"114584312,00",01.2013,19-24,M,Samotność,NaN,NaN,NaN,1.0,NaN,Inny,NaN,NaN,NaN,2013,01,1
4,"114584399,00",01.2013,40-44,M,Samotność,NaN,Bezrobotny,NaN,1.0,Zabudowania,Powieszenie,Alko,NaN,NaN,2013,01,0


#Zapis

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')
# df_raw.to_csv('/content/drive/My Drive/preped_samobojstwa_2013_2022.csv', index=False)
# df_powód_zamachu.to_csv('/content/drive/My Drive/preped_powod_zamachu_samobojstwa_2013_2022.csv', index=False)

In [ ]:
file_name = 'preped_samobojstwa_2013_2022.csv'
df_raw.to_csv(file_name, index=False)
files.download(file_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
file_name = 'preped_powod_zamachu_samobojstwa_2013_2022.csv'
df_powód_zamachu.to_csv(file_name, index=False)
files.download(file_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>